# Install

In [1]:
!git clone https://github.com/nvidia/pix2pixhd

Cloning into 'pix2pixhd'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 55.68 MiB | 2.95 MiB/s, done.
Resolving deltas: 100% (156/156), done.


In [10]:
import os
os.chdir('/home/jovyan/ndvi_p2p/pix2pixhd')

In [11]:
!pip install dominate

In [15]:
# fix train.py to have math.gcd instead of fractions on line 9
# In data/aligned_dataset.py script, use ".convert('L') or .convert('1')" for loading B images.
# In data/aligned_dataset.py script, set normalize =False for the B dataset
# In data/aligned_dataset.py scriptm, add your own 1 channel normlaization
# in line 192 models/pix2pixHDmodel.py
        if not self.opt.no_vgg_loss:
            if self.opt.output_nc == 1: # for output_nc == 1
                image_shape = fake_image.shape
                fake_image3 = torch.zeros([image_shape[0],3,image_shape[2],image_shape[3]])
                real_image3 = torch.zeros([image_shape[0],3,image_shape[2],image_shape[3]])
                fake_image3[:,0,:,:] = fake_image3[:,1,:,:] = fake_image3[:,2,:,:] = fake_image
                real_image3[:, 0, :, :] = real_image3[:, 1, :, :] = real_image3[:, 2, :, :] = real_image
                loss_G_VGG = self.criterionVGG(fake_image3.cuda(), real_image.cuda()) * self.opt.lambda_feat
            else:
                loss_G_VGG = self.criterionVGG(fake_image, real_image) * self.opt.lambda_feat

# Datasets

Training with your own dataset

If your input is not a label map, please just specify --label_nc 0 which will directly use the RGB colors as input. The folders should then be named train_A, train_B instead of train_label, train_img, where the goal is to translate images from A to B.
If you don't have instance maps or don't want to use them, please specify --no_instance.
The default setting for preprocessing is scale_width, which will scale the width of all training images to opt.loadSize (1024) while keeping the aspect ratio. If you want a different setting, please change it by using the --resize_or_crop option. For example, scale_width_and_crop first resizes the image to have width opt.loadSize and then does random cropping of size (opt.fineSize, opt.fineSize). crop skips the resizing step and only performs random cropping. If you don't want any preprocessing, please specify none, which will do nothing other than making sure the image is divisible by 32.

NEED BIGGER IMGS OFCCCC


In [6]:
#Prepare the p2p dataset
dataroot = "/home/jovyan/ndvi_p2p/p2pHD_msrgb"

# restructued A/traintestval to train_A, val_A, etc.etc. through the terminal/cli

# Training

-   `python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction BtoA`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. Add `--direction BtoA` if you want to train a model to transfrom from class B to A.

In [24]:
!python train.py --dataroot {dataroot} \
  --name rgb_ndvi_p2pHD100 \
  --model pix2pixHD \
  --output_nc 1 \
  --batchSize 1 \
  --loadSize 512 \
  --no_instance \
  --label_nc 0 \
  --data_type 8 \
  --resize_or_crop none \
  --niter 50 \
  --niter_decay 50 \
  --no_vgg_loss 

------------ Options -------------
batchSize: 1
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: False
data_type: 8
dataroot: /home/jovyan/ndvi_p2p/p2pHD_msrgb
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 0
lambda_feat: 10.0
loadSize: 512
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: rgb_ndvi_p2pHD100
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 50
niter_decay: 50
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: True
no_lsgan: False
no_vgg_loss: True
norm: instance
num_D: 2
output_nc: 1
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: none
save_epoch_freq: 10
save_latest_freq: 1000
serial_batches: False
t

# Testing

-   `python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_pix2pix`

Change the `--dataroot`, `--name`, and `--direction` to be consistent with your trained model's configuration and how you want to transform images.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> Note that we specified --direction BtoA as Facades dataset's A to B direction is photos to labels.

> If you would like to apply a pre-trained model to a collection of input images (rather than image pairs), please use --model test option. See ./scripts/test_single.sh for how to apply a model to Facade label maps (stored in the directory facades/testB).

> See a list of currently available models at ./scripts/download_pix2pix_model.sh

In [12]:
!ls checkpoints/
dataroot = "/home/jovyan/ndvi_p2p/canyelles_512"

rgb_ndvi_p2pHD	rgb_ndvi_p2pHD100


In [13]:
!python test.py --dataroot {dataroot} \
  --name rgb_ndvi_p2pHD100 \
  --model pix2pixHD \
  --output_nc 1 \
  --batchSize 1 \
  --loadSize 512 \
  --no_instance \
  --label_nc 0 \
  --resize_or_crop none \
  --how_many 1000 \
  --phase test


------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: /home/jovyan/ndvi_p2p/canyelles_512
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 1000
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 512
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: rgb_ndvi_p2pHD100
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: True
norm: instance
ntest: inf
onnx: None
output_nc: 1
phase: test
resize_or_crop: none
results_dir: ./results/
serial_batches: False
tf_log: False
use_dropout: False
use_encoded_image: False
verbose: False
which_epoch: latest
-------------- End ----------------
CustomDatasetDataLoader
data

In [10]:
!python test.py --dataroot {dataroot} \
  --name rgb_ndvi_p2p \
  --model pix2pix \
  --output_nc 1 \
  --direction AtoB \
  --batch_size 32 \
  --load_size 256 \
  --dataset_mode aligned \
  --epoch 100 \
  --num_test 50000 \
  --phase test

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 32                            	[default: 1]
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: /home/jovyan/ndvi_p2p/data    	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: 100                           	[default: latest]
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter

# Visualize

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_fake_B.png')
plt.imshow(img)

In [ ]:
img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_real_A.png')
plt.imshow(img)

In [ ]:
img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_real_B.png')
plt.imshow(img)